In [2]:
def convert_dict(fname):#this function is to convert json dictionary into a python dictionary and do little bit of text-cleaning   
    doc=[]
    result_title=[]
    result_text=[]
    ans_text=[]
    ball=[]

    with open(fname, 'r') as fi:
        for i in fi:
            null=None#as there are null values in the json dictionary, so converting them as None
            doc.append(eval(i))#converting a json dict into a python dict. tried to do this conversion using ast.literal_eval which is much safer than eval since it will only evaluate strings which contain literals
            #but that posed an error because of "string" literal in the json dictionary
    for ele in doc:#get the values from the python dictionary, doc and appending the values to two separate lists 
        if "text" in ele:
            result_text.append(str(ele["text"]).lower().replace("\n", "").replace("/", ""))#getting values of the dictionary key: text and appending it to result_text
        if "title" in ele:
            result_title.append(str(ele["title"]).lower().replace("\n", "").replace("/", ""))#getting values of the dictionary key: title and appending it to result_title
    return result_text, result_title

def stop_lemmatize(rtext):#this function comprises preprocessing steps re cleaning the text: first tokenize after removing punctuation, second remove insignificant words that can be found in stopwords
    #third use WordNetLemmatizer to get common base form of words. I ended up doing all these steps manually as the CountVectorizer do not have lemmatization in play
    from nltk.tokenize import word_tokenize
    from nltk.stem.wordnet import WordNetLemmatizer
    from nltk.corpus import stopwords
    import string
    rtext = [''.join(c for c in s if c not in string.punctuation) for s in rtext]#removing punctuation
    docs = [word_tokenize(content) for content in rtext]#tokenizing the texts into words
    stop = set(stopwords.words('english'))
    wordnet = WordNetLemmatizer()
    doc_fair = [[word for word in words if word not in stop] for words in docs]#removing words that are in stopwords
    docs_w = [' '.join(wordnet.lemmatize(w1).replace("‘", '').replace("’", '').replace("'", '') for w1 in w2) for w2 in doc_fair]#lemmatizing to get a common base form of words. finally joining the words to get a neat list of strings to pass in to the CountVectorizer
    return docs_w

def final_lists(f_name):
    result_text, result_title = f_name
    docs_wordnet=stop_lemmatize(result_text)
    docs_wordnet_title=stop_lemmatize(result_title)
    return docs_wordnet, docs_wordnet_title

In [3]:
#assigning the outcome of the above function: final_lists to a dataframe columns for an easy access to get the titles to associate with the clusters to get an idea 
#on how the algorithm have grouped the texts
df1=pd.DataFrame()
df1['text_document'], df1['title']=final_lists(convert_dict("/Users/user_name/Downloads/corpus.txt"))
                                               

In [4]:
df1.head()

,text_document,title
0,joe raedlegetty imageslast week wrote story sl...,reason skeptical teslapowered puerto rico
1,zach boydenholmesthe registerpolice identified...,police threat west de moines student work copycat
2,chicago police investigating whether stephen p...,gunman reserved two room blackstone
3,nashville tenn 5 georgia coming two impressive...,5 georgia bulldog vanderbilt commodore preview...
4,,craig robinson adam scott buddy fox supernatur...


In [84]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
n_features =15000

tf_v = CountVectorizer(max_df=0.95, min_df=1, max_features=n_features, lowercase=False)#lowercase=False as the texts were converted into lowercase in the above steps
tf_x = tf_v.fit_transform(df1['text_document'])#to get X_values


In [85]:
#To get y_values - just for labelling
tf_y = tf_v.fit_transform(df1['title'])

In [62]:
#Tried to split the datset into train and test and compare how well the model does. But did not have time to get this to work'
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(tf_x, tf_y, test_size=0.20, random_state=42)

In [65]:
#To fit LDA on X_train
from sklearn.decomposition import LatentDirichletAllocation
n_components = 60
lda = LatentDirichletAllocation(n_components=n_components, doc_topic_prior=0.83, topic_word_prior=0.1, learning_method='online', learning_offset=50.,random_state=0)
lda1=lda.fit(X_train)

In [78]:
def print_top_words(model, feature_names, n_top_words):#this function finds the top words(taken from sklearn)
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i] for i in topic.argsort()[:-n_top_words -1:-1]])
        print(message)
    print()

In [79]:
#The top 20 words in 60 clusters using X-train
n_top_words = 20
#print("\nTopics in LDA model:")
tf_feature_names = tf_v.get_feature_names()
# len(tf_feature_names)
print_top_words(lda1, tf_feature_names, n_top_words)

Topic #0: enbd succeed collect postpones pot subvert link simcyp unwittingly dastyaris wiley hitting poof affecting p20 achieve coldshooting shaking buildadapp plaza
Topic #1: pioneering krishna bezel simcyp telegram persisted taxpayer affecting wiley welding experience upgraded trumpcard positek provider yoyo dennis concussion patient pathway
Topic #2: laptop simcyp booming taxpayer expectation pageant mahoney yoyo wrestlemania trumpcard papal eelo persisted men positek benghazi petal upgraded dastyaris poof
Topic #3: pinpoint aggravated defenseman elías louisiana satriani shaking remitly peru seeing waterford chides marcus affecting stable troy soundtrack ntp yoyo simcyp
Topic #4: biometric politics simcyp taxpayer hitting qualifying persisted turbine yearender apra positek midfielder upgraded yoyo affecting trumpcard coldshooting net rolet mahoney
Topic #5: slaughterbots affecting poof taxpayer upgraded provider positek ago yearender tipped simcyp mahoney persisted yoyo qualifying p

In [80]:
#now on to try the model's performance on X_test:
lda2=lda.fit(X_test)

In [83]:
#The top 20 words in 60 clusters using X-test
n_top_words = 20
#print("\nTopics in LDA model:")
tf_feature_names_test = tf_v.get_feature_names()
# len(tf_feature_names)
print_top_words(lda2, tf_feature_names_test , n_top_words)

Topic #0: dad taxpayer simcyp inch persisted establish affecting yoyo assassination positek dyer orrin yearender authority plea sixmonth realizing canadiens qualifying ecstatic
Topic #1: aussie brill didnt weekday showdown taxpayer 57000 simcyp persisted positek patras choudhary affecting canadiens plea yoyo willie ackman assassination undetermined
Topic #2: taxpayer simcyp positek canadiens upgraded yoyo roof plea trulia willie clemsons persisted spring nissan ackman coo nursery yandex dyer dwayne
Topic #3: 6game jonathan dcoordinator tinto approach 800k accrington nonprofit norfolk calgary luggage indictment eos burned prosthetic timber icloud pueblo toughest confrontation
Topic #4: alarm establish taxpayer simcyp persisted midfielder qualifying yoyo positek clemsons sworn undetermined 2018 sixmonth yearender hazing superfan coldshooting trumpcard swope
Topic #5: undetermined taxpayer simcyp nursery suspended superfan scientific willie dwayne spring 57000 plea badgerys ackman bitpay 